<a href="https://colab.research.google.com/github/joony0512/Deep_Learning_Class/blob/main/Part5/P5_Ch06_CH06_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM seq2seq

## DataLoader 만들기

In [1]:
!pip install portalocker

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
import sys
from datetime import datetime

drive_project_root = '/content/drive/MyDrive/#fastcampus'
sys.path.append(drive_project_root)
!pip install -r '/content/drive/MyDrive/#fastcampus/requirements.txt'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  Using cached tensorflow_addons-0.20.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (591 kB)
  Using cached onnx-1.10.2.tar.gz (9.9 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached onnx_tf-1.9.0-py3-none-any.whl (222 kB)
  Using cached tf2onnx-1.9.3-py3-none-any.whl (435 kB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached typeguard-2.13.3-py3-none-any.whl (17 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for onnx (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for onnx
Failed to build onnx
ERROR:

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >=0 :
    print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
    print('and then re-execute this cell')
else :
    print(gpu_info)

Tue Jul 11 11:11:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install omegaconf
!pip install torch_optimizer
!pip install wandb
!pip install efficientnet_pytorch==0.7.1
!pip install hydra-core==1.1
!pip install pytorch-lightning
!pip install --upgrade torchmetrics
!pip install --upgrade pytorch-lightning

In [34]:
# for data loading
from typing import List
from typing import Dict
from typing import Union
from typing import Any
from typing import Optional
from typing import Iterable
from typing import Callable
from abc import abstractmethod
from abc import ABC
from datetime import datetime
from functools import partial
from collections import Counter
from collections import OrderedDict
import random
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pprint import pprint

from torchtext import data
from torchtext import datasets
from torchtext.datasets import Multi30k
from torchtext.data.utils import get_tokenizer
from torchtext.data.functional import to_map_style_dataset
from torchtext.vocab import Vocab, build_vocab_from_iterator, vocab
import spacy

# for contiguration
from omegaconf import DictConfig
from omegaconf import OmegaConf
import hydra
from hydra.core.config_store import ConfigStore

# for logger
from torch.utils.tensorboard import SummaryWriter
import wandb
os.environ["WANDB_START_METHOD"] ="thread"

In [6]:

%cd /content/drive/MyDrive/#fastcampus
from data_utils import dataset_split
from config_utils import flatten_dict
from config_utils import register_config
from config_utils import configure_optimizers_from_cfg
from config_utils import get_loggers
from config_utils import get_callbacks
%cd /content


/content/drive/MyDrive/#fastcampus
/content


In [ ]:
# download spacy data
!python -m spacy download en
!python -m spacy download en_core_web_sm
!python -m spacy download de
!python -m spacy download de_core_news_sm


In [22]:
data_spacy_de_en_cfg = {
    "name" : "spacy_de_en",
    "data_root" : os.path.join(os.getcwd(), "data"),
    "tokenizer" : "spacy",
    "src_lang" : "de",
    "tgt_lang" : "en",
    "src_index" : 0,
    "tgt_index" : 1,
    "vocab" :{
        "special_symbol2index" :{
            "<unk>" : 0, # 모르는 단어 처리
            "<pad>" : 1, # 길이 다른것 패딩으로 처리
            "<bos>" : 2, # 끝부분 표시
            "<eos>" : 3, # 첫부분 표시
        } ,
        "special_first" : True, # 위의 것들을 맨처음으로 배치
        "min_freq" : 2,
    }
}

data_cfg = OmegaConf.create(data_spacy_de_en_cfg)
print(OmegaConf.to_yaml(data_cfg))

name: spacy_de_en
data_root: /content/data
tokenizer: spacy
src_lang: de
tgt_lang: en
src_index: 0
tgt_index: 1
vocab:
  special_symbol2index:
    <unk>: 0
    <pad>: 1
    <bos>: 2
    <eos>: 3
  special_first: true
  min_freq: 2



In [9]:
train_data, valid_data, test_data = Multi30k(data_cfg.data_root)
test_data = to_map_style_dataset(test_data)

In [ ]:
for i in test_data :
    print(i)

In [16]:
# 1. token_transform ( token ... )
def get_token_transform(data_cfg : DictConfig) -> dict :
    token_transform : dict ={}
    token_transform[data_cfg.src_lang] = get_tokenizer(
        data_cfg.tokenizer, language = data_cfg.src_lang
        )
    token_transform[data_cfg.tgt_lang] = get_tokenizer(
        data_cfg.tokenizer, language = data_cfg.tgt_lang
        )
    return token_transform

token_transform = get_token_transform(data_cfg)

/usr/local/lib/python3.10/dist-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(


In [23]:
# 2. vocab_transform

def yeild_tokens(
    data_iter : Iterable, lang : str, lang2index : Dict[str, int],
) -> List[str] :
    """help function to yeild list of tokens"""
    for data_sample in data_iter:
        yield token_transform[lang](data_sample[lang2index[lang]])

def get_vocab_transform(data_cfg : DictConfig) -> dict :
    vocab_transform : dict = {}
    for ln in [data_cfg.src_lang, data_cfg.tgt_lang]:
        # build from train_data
        train_iter = Multi30k(
            split = 'train', language_pair =(data_cfg.src_lang, data_cfg.tgt_lang)
        )
        # create torchtext's Vocab object
        vocab_transform[ln] =build_vocab_from_iterator(
            yeild_tokens(
                train_iter,
                ln,
                {
                    data_cfg.src_lang : data_cfg.src_index,
                    data_cfg.tgt_lang : data_cfg.tgt_index,
                }
            ),
            min_freq = data_cfg.vocab.min_freq,
            specials = list(data_cfg.vocab.special_symbol2index.keys()),
            special_first = True
        )

    # set UNKNOWM as the default index, --> index가 unknown으로 return : token이 찾아지지 않을 경우
    # 만약 세팅되지 않으면 runtime error가 날 수 있다.
    for ln in [data_cfg.src_lang, data_cfg.tgt_lang]:
        vocab_transform[ln].set_default_index(data_cfg.vocab.special_symbol2index["<unk>"])
    return vocab_transform

vocab_transform = get_vocab_transform(data_cfg)


In [27]:
print(vocab_transform['de']['<bos>'])
print(vocab_transform['en']['<bos>'])
print(vocab_transform['en']['hello'],vocab_transform['en']['world'])

2
2
5465 1870


In [29]:
# 3. integrated transforms
# --> text_transform : [token_transform -> vocab_transform -> torch.tensor transform]

# helper function for collate_fn

def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms :
            txt_input = transform(txt_input)
        return txt_input
    return func

# convert to torch.tensor with bos & eos
def tensor_transform(token_ids : List[int], bos_index : int, eos_index : int ):
    return torch.cat(
        (torch.tensor([bos_index]),torch.tensor(token_ids) ,torch.tensor([eos_index]))
        )

# src & tgt lang language text_transforms to convert raw strings --> tensor indices
def get_text_transform(data_cfg: DictConfig):
    text_transform = {}
    for ln in [data_cfg.src_lang, data_cfg.tgt_lang]:
        text_transform[ln] = sequential_transforms(
            token_transform[ln],
            vocab_transform[ln],
            partial(
                tensor_transform,
                bos_index = data_cfg.vocab.special_symbol2index["<bos>"],
                eos_index = data_cfg.vocab.special_symbol2index["<eos>"],
            )
        )
    return text_transform

text_transform = get_text_transform(data_cfg)

In [33]:
print(text_transform["en"]('hello'))
print(text_transform["en"]('hello,'))
print(text_transform["en"]('hello, how'))
print(text_transform["en"]('hello, how are you ?'))

tensor([   2, 5465,    3])
tensor([   2, 5465,   15,    3])
tensor([   2, 5465,   15,  889,    3])
tensor([   2, 5465,   15,  889,   17, 1328, 2470,    3])


In [37]:
# 4. collate_fn --> batch를 전처리 할까?

def collate_fn(batch, data_cfg: DictConfig):
    src_batch , tgt_batch =[], []

    for src_sample, tgt_sample in batch :
        src_batch.append(text_transform[data_cfg.src_lang](src_sample.rstrip('\n')))
        tgt_batch.append(text_transform[data_cfg.tgt_lang](tgt_sample.rstrip('\n')))

    src_batch = pad_sequence(src_batch, padding_value= data_cfg.vocab.special_symbol2index['<pad>'])
    tgt_batch = pad_sequence(tgt_batch, padding_value= data_cfg.vocab.special_symbol2index['<pad>'])
    return src_batch, tgt_batch

def get_collate_fn(cfg : DictConfig):
    return partial(collate_fn, data_cfg = cfg.data)

# 5. DataLoader

def get_multi30k_dataloader(split_mode : str , language_pair : tuple, batch_size : int, collate_fn: Callable):

    iter = Multi30k(split=split_mode, language_pair =language_pair)
    dataset = to_map_style_dataset(iter)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size =batch_size , collate_fn = collate_fn )
    return dataloader


test_dataloader =get_multi30k_dataloader('test', (data_cfg.src_lang, data_cfg.tgt_lang), 3, collate_fn = partial(collate_fn, data_cfg = data_cfg))

In [38]:
for i in test_dataloader :
    print(i)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
        [876,   1,   1],
        [  5,   1,   1],
        [  3,   1,   1]]))
(tensor([[   2,    2,    2],
        [  14,  126,   21],
        [  17,   17,  114],
        [   0,   22,  123],
        [   0,   51, 2455],
        [   7,  132,    6],
        [   6,  302, 2056],
        [   0,   10,   33],
        [   3,   26,   10],
        [   1,   12,    6],
        [   1,   22,  512],
        [   1,  256,    3],
        [   1,    4,    1],
        [   1,    3,    1]]), tensor([[   2,    2,    2],
        [   6,   49,   19],
        [  16,   16,   77],
        [  10,    7,  117],
        [   0,    8, 2826],
        [1193,   31,    4],
        [5374,  119, 1927],
        [   7,   10,   34],
        [   4,  241,   14],
        [2120,   14,    4],
        [1623,    8,  215],
        [   3,   12,   69],
        [   1,    7,    3],
        [   1,    4,    1],
        [   1,  200,    1],
        [   1,    5,    1],
        [   1,    3,    1]]))
(tensor([[   2